In [4]:
from textblob import TextBlob
import pandas as pd
pd.set_option('display.max_rows', 200)

cleaned_proposals = pd.read_parquet("../output_data/snapshot/cleaned_proposals.parquet")
forum_topics = pd.read_parquet("../output_data/forum/topics.parquet")

proposal_merged_df = pd.merge(
    cleaned_proposals, forum_topics, left_on="forum_topic_id", right_on="id", suffixes=('_proposal', '_topic')
)

posts = pd.read_parquet("../output_data/forum/posts.parquet")


merged_df = pd.merge(
    proposal_merged_df, posts, left_on="forum_topic_id", right_on="topic_id", how="inner", suffixes=('_', '_post')
)
df = merged_df[["id_proposal", "title_proposal", "votes", "scores_total", "selected_choice", "forum_topic_id", "id", "raw"]]
print(df.info())
# print(merged_df[merged_df["topic_id"] == 20957]["raw"])


# column_names_list = list(merged_df.columns)
# column_names_list

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 4834 entries, 0 to 4833
Data columns (total 8 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   id_proposal      4834 non-null   object 
 1   title_proposal   4834 non-null   object 
 2   votes            4834 non-null   int64  
 3   scores_total     4834 non-null   float64
 4   selected_choice  4834 non-null   object 
 5   forum_topic_id   4834 non-null   int64  
 6   id               4834 non-null   int64  
 7   raw              4834 non-null   object 
dtypes: float64(1), int64(3), object(4)
memory usage: 302.3+ KB
None


In [57]:
# TODO: add data cleaning(remove html tag)

# Apply sentiment analysis to each record and split the results into two columns: 'polarity' and 'subjectivity'
df[['polarity', 'subjectivity']] = df['raw'].apply(lambda x: pd.Series(TextBlob(x).sentiment))

sentiment_by_proposal = df.groupby(['id_proposal', 'title_proposal', "votes", "scores_total", "selected_choice", "forum_topic_id"])[['polarity', 'subjectivity']].mean().reset_index()

sentiment_by_proposal.to_csv("data/sentiment_by_proposal.csv", index=False)
# low = average_polarity[average_polarity['polarity'] == average_polarity['polarity'].min()]



# df.head()
# average_polarity = df['polarity'].mean()
# print(average_polarity)
# average_subjectivity = df['subjectivity'].mean()
# print(average_subjectivity)


/var/folders/gl/3nbwj5r1183_gfp3xql0hclr0000gn/T/ipykernel_8133/2931585712.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df[['polarity', 'subjectivity']] = df['raw'].apply(lambda x: pd.Series(TextBlob(x).sentiment))


## Visualize 

In [26]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px
from scipy.stats import ttest_ind

df = pd.read_csv('data/sentiment_by_proposal.csv')

df = df[df["selected_choice"].isin(['For', 'Against', 'Abstain'])]


figPol = px.box(df,x='selected_choice', y="polarity")
figPol.show()

forGroup = df[df['selected_choice'] == 'For']
againstGroup = df[df['selected_choice'] == 'Against']

result = ttest_ind(forGroup['polarity'], againstGroup['polarity'], equal_var=False)
print(result)
# p = 0.0001778433733127446

# Create a boxplot
figSub = px.box(df,x='selected_choice', y="subjectivity")
figSub.show()
result = ttest_ind(forGroup['subjectivity'], againstGroup['subjectivity'], equal_var=False)
print(result)
# p = 0.0001778433733127446



TtestResult(statistic=3.9771550458754175, pvalue=0.0001778433733127446, df=65.02940141544285)


TtestResult(statistic=-0.17036940302998288, pvalue=0.8654845543059526, df=44.92732802358942)


In [19]:
import plotly.express as px

df = pd.read_csv('data/sentiment_by_proposal.csv')

df = df[df["selected_choice"].isin(['For', 'Against', 'Abstain'])]

figPol = px.scatter(df, x='polarity', y='votes', hover_name='title_proposal', trendline='ols', trendline_color_override='darkblue', title="Total votes by polarity")


figPol.show()

figSub = px.scatter(df, x='subjectivity', y='votes', hover_name='title_proposal', trendline='ols', trendline_color_override='darkblue', title="Total votes by subjectivity")

figSub.show()

In [20]:

df_for_corr = df[['polarity', 'votes', 'subjectivity', 'scores_total']]
df_corr = df_for_corr.corr()
print(df_corr)

# r
# polarity <> votes = 0.22
# subjectivity <> votes = -0.3

              polarity     votes  subjectivity  scores_total
polarity      1.000000  0.226253      0.226363      0.055215
votes         0.226253  1.000000     -0.306806     -0.283331
subjectivity  0.226363 -0.306806      1.000000      0.150402
scores_total  0.055215 -0.283331      0.150402      1.000000
